In [ ]:
# fonction de téléchargement des données sur les accidents corporels
def telecharge(url_data,filename, path):
    # Vérifie si le dossier 'data' existe, sinon le crée
    if not os.path.exists(path):
        os.makedirs(path)
    #télécharge les données avec l'url
    response = requests.get(url_data)
    if response.status_code == 200:
        file_path = os.path.join(path, filename)
        with open(file_path, 'wb') as file:
            file.write(response.content)
        print(f"Fichier {filename} téléchargé avec succès:{file_path}")
    else:
        print(f"Echec de téléchargement pour {filename}. Statut: {response.status_code}")

# API pour accéder à l'url de téléchargement
url_root="https://www.data.gouv.fr/api/1/datasets/53698f4ca3a729239d2036df/resources/"
urls={
    "usagers-2023.csv":"68848e2a-28dd-4efc-9d5f-d512f7dbe66f",
    "vehicules-2023.csv":"146a42f5-19f0-4b3e-a887-5cd8fbef057b",
    "lieux-2023.csv":"8bef19bf-a5e4-46b3-b5f9-a145da4686bc",
    "caract-2023.csv":"104dbb32-704f-4e99-a71e-43563cb604f2"
}
path='/home/onyxia/Projet-Python-pour-la-Data-Science/data'

for filename, resource_id in urls.items():
    url=url_root+resource_id
    response1=requests.get(url)
    if response1.status_code==200:
        data=response1.json()
        url_data=data['url']
    else:
        print("downloading failed")
    telecharge(url_data,filename,path)


In [ ]:
# base des usagers 
df_usagers = pd.read_csv(r"/home/onyxia/Projet-Python-pour-la-Data-Science/data/usagers-2023.csv", sep = ';')
df_usagers.head()

In [ ]:
# Base des véhicules 
df_vehicules = pd.read_csv(r"/home/onyxia/Projet-Python-pour-la-Data-Science/data/vehicules-2023.csv", sep = ';')
df_vehicules.head()

In [ ]:
# Base des lieux 
df_lieux = pd.read_csv(r"/home/onyxia/Projet-Python-pour-la-Data-Science/data/lieux-2023.csv", sep = ';')
df_lieux.head(10)

In [ ]:
df_caract = pd.read_csv(r"/home/onyxia/Projet-Python-pour-la-Data-Science/data/caract-2023.csv", sep = ';')
df_caract.head()

In [ ]:
df_merge = df_usagers.merge(df_vehicules, on=["Num_Acc","id_vehicule","num_veh"], how="inner") 
df_merge = df_merge.merge(df_lieux, on="Num_Acc", how="inner")
df_merge = df_merge.merge(df_caract, on="Num_Acc", how="inner")
df_merge.shape

In [ ]:
### Modélisation

In [ ]:
# Packages nécéssaire
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler


In [ ]:
# calcul de l'âge des piétons
df_merge["Age"] = 2023-df_merge["an_nais"]
# Distribution de l'âge
df_merge["Age"].hist()

In [ ]:
#supression des variables d'identification
var=["Num_Acc","jour","an","com","adr","lat","long","voie","v1","v2","id_vehicule","num_veh","id_usager","an_nais", "dep"]
Num_acc=df_merge["Num_Acc"]
df_merge.drop(var, axis=1,inplace=True)

# pourcentage des valeurs manquantes par variables 
df_merge.isna().sum()[df_merge.isna().sum()!=0]/len(df_merge)*100

In [ ]:
df_merge.drop(["occutc","lartpc"], axis=1, inplace=True)

In [ ]:
# imputation des valeurs manquantes de l'âge
sns.boxplot(data=df_merge, x=df_merge['Age'])

In [ ]:
df_merge["Age"]=df_merge["Age"].fillna(df_merge["Age"].median())

In [ ]:
#recodage des variables trajet et actp de la base usager
df_merge["trajet"]=df_merge["trajet"].replace(0, value=-1)
df_merge["actp"]=df_merge["actp"].replace(0, value=-1)

# variables de type object de la base
df_merge.select_dtypes("object").columns

In [ ]:
# conversion des variables de type object en variables numériques
def replace_pr(val):
    if len(val.split())==1:
        val=val
    else:
        if len(val.split())>1:
            val=val.split()[1]
        else:
            val="-1"
    return val
df_merge["pr"]=df_merge["pr"].apply(func=replace_pr)
df_merge["pr"]=df_merge["pr"].astype(float)
df_merge["pr1"]=df_merge["pr1"].apply(func=replace_pr)
df_merge["pr1"]=df_merge["pr1"].astype(float)
def rep_virgule(val):
    val=val.replace(",",".")
    return val
df_merge["larrout"]=df_merge["larrout"].apply(rep_virgule)
df_merge["larrout"]=df_merge["larrout"].astype(float)
def heure(val):
    val=val.split(":")[0]
    return val
df_merge["hrmn"]=df_merge["hrmn"].apply(func=heure)
df_merge["hrmn"]=df_merge["hrmn"].astype(float)

df_merge["nbv"]=df_merge["nbv"].str.strip()
df_merge["nbv"]=df_merge["nbv"].replace("#VALEURMULTI","-1")
df_merge["nbv"]=df_merge["nbv"].astype(float)

df_merge["actp"]=df_merge["actp"].replace(["A","B"],["10","11"])
df_merge["actp"]=df_merge["actp"].astype(float)

In [ ]:
#suppression des colonnes qui ont plus de 20% de valeurs non renseignés
diction=[]
for column in df_merge.columns:
    if ((len(df_merge[df_merge[column]==-1][column])/df_merge.shape[0]*100)>=20) == True:
        diction.append(column)
        
df_merge=df_merge.drop(diction, axis=1)

In [ ]:
# on remplace toute les valeurs non renseignées des variables restantes par le mode
diction2=[]
for column in df_merge.columns:
    if len(df_merge[df_merge[column]==-1][column])/df_merge.shape[0]!=0:
        diction2.append(column)

for col in diction2:
    mode=df_merge[col].value_counts().idxmax()
    df_merge[col]=df_merge[col].replace(-1,mode)


In [ ]:
#On caractérise chaque accident par le niveau de gravité le plus haut
df_merge["Num_Acc"]=Num_acc
import numpy as np
id_acc=df_merge["Num_Acc"].unique()
index_true=[]
for i in id_acc:
    df_id=df_merge[df_merge["Num_Acc"]==i]
    n_grav=np.max(df_id["grav"])
    index=df_merge[(df_merge["Num_Acc"]==i) & (df_merge["grav"]==n_grav)].index[0]
    #if len(index)!=0:
    #df_merge.drop(index=index, inplace=True)
    index_true.append(index)
        
df_merge.drop(["Num_Acc"], axis=1, inplace=True)

In [ ]:
# Niveau de gravité de l'accident présente dans la nouvelle base
df_merge=df_merge.loc[index_true]
df_merge["grav"].value_counts()

Dans les accidents enregistrés, aucun n'a laisser toutes les personnes impliquées indemnes.

On recode la variable "gravité" par: 
- 2: niveau de gravité **grave** (2)
- 3: niveau de gravité **moyen** (1)
- 4: niveau de gravité **faible** (0)

In [ ]:
#recodage de la gravité de l'accident
df_merge["grav"]=df_merge["grav"].replace([3,4], value=[1,0])
# distribution de la gravité
df_merge["grav"].value_counts()/df_merge.shape[0]*100

#### Choix des variables

In [ ]:
#corrélation entre les variables
plt.figure(figsize=(20,20))
sns.heatmap(df_merge.corr(),annot=True)


In [ ]:
# on retire les variables qui ont une forte corrélation avec d'autres
df_merge=df_merge.drop(["place","agg"], axis=1)

In [ ]:
# en utilisant une regression lasso
numeric_var=["Age",'hrmn','vma']
lasso_x=df_merge.drop(["grav"],axis=1)
categorical_var=lasso_x.drop(numeric_var, axis=1).columns
lasso_y=df_merge["grav"]

numeric_pipeline = Pipeline(
    steps=[("scale", StandardScaler())]
)

categorical_pipeline = Pipeline(
    steps=[
        ("one-hot", OneHotEncoder(handle_unknown="ignore", sparse_output=False)),
    ]
)

base=categorical_pipeline.fit_transform(lasso_x.drop(columns=numeric_var,axis=1))
base_cont=numeric_pipeline.fit_transform(lasso_x[numeric_var])
data=pd.DataFrame(data=base, columns=categorical_pipeline.get_feature_names_out())
data[numeric_var]=base_cont

my_alphas = np.array([0.001, 0.01, 0.02, 0.025, 0.05, 0.1, 0.25, 0.5, 0.8, 1.0])

lcv = LassoCV(alphas=my_alphas, fit_intercept=False, random_state=0, cv=5).fit(
    data, lasso_y
)
print("alpha optimal :", lcv.alpha_)


In [ ]:
# Application du lasso
model = Lasso(fit_intercept=False, alpha=lcv.alpha_)
lasso_optimal=model.fit(data, lasso_y)
data.columns[np.abs(lasso_optimal.coef_)>0] # variables sélectionner

In [ ]:
data_model=data[data.columns[np.abs(lasso_optimal.coef_)>0]]

#### Implémentation de la régression logistique

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(data_model,lasso_y, test_size=0.2, random_state=42)

In [ ]:
params={'penalty':["l2",None], 'solver':["newton-cg"]}
cv=RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
grid= GridSearchCV(LogisticRegression(multi_class="multinomial",class_weight="balanced"), params, cv=cv)
grid.fit(x_train, y_train)
print("best params:",grid.best_params_)
print("best score:",grid.best_score_)


In [ ]:
model=grid.best_estimator_
acc_train = accuracy_score(y_train, model.predict(x_train))
acc_test = accuracy_score(y_test, model.predict(x_test))
print("Training Accuracy:", round(acc_train, 2))
print("Test Accuracy:", round(acc_test, 2))

In [ ]:
plt.figure(figsize=(12,12))
importances1 = model.coef_[0]
odds_ratios1 = pd.Series(np.exp(importances1), index=data_model.columns).sort_values() #coefficient des variables dans le modèle
plt.subplot(1,2,1)
odds_ratios1.tail(15).plot(kind="barh")
plt.ylabel("Odds Ratio")

importances2 = model.coef_[1]
odds_ratios2 = pd.Series(np.exp(importances2), index=data_model.columns).sort_values() #coefficient des variables dans le modèle
plt.subplot(1,2,2)
odds_ratios2.tail(15).plot(kind="barh")
plt.ylabel("Odds Ratio")

- L'utilisation de la ceinture de sécurité seule augmente la chance que la ravité de l'accident soit moyenne de 82%.
- Lorsque la voiture se trouve entre deux files avant l'accident, la gravité de l'accident à 54% d'être moyenne que d'être faible
- un véhicule scooter<50cm3, l'autoroute, heurté une glissière en béton augmentent la chance d'avoir un accident de gravité moyenne de 54%,51%,47% respectivement.
- une collision de trois véhicules et plus-en chaine augmentent la chance d'avoir un accident de gravité moyenne de 45%.
 - les caractéristiques suivantes: une manoeuvre d'évitement, un accident sur piste cyclable, heurté un véhicule en stationnement, avec un scooter > 50cm3 et <=125cm3, avoir un accident sur la chaussée, ou avec un véhicule léger, ou encore sous une pluie légère,  augmentent la chance d'avoir un accident de gravité moyenne entre 15% et 30%. 

 - Avoir un accident sur un cyclomoteur<50cm3 ou sur une motocyclette augmente la probabilité que l'accident soit grave de plus de 40%.

In [ ]:
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
odds_ratios1.head(10).plot(kind="barh")
plt.ylabel("Odds Ratio")
plt.subplot(1,2,2)
odds_ratios2.head(10).plot(kind="barh")
plt.ylabel("Odds Ratio")